In [1]:
# Si pas encore installé
# !pip install transformers datasets torch --break-system-packages

from transformers import CamembertTokenizer, CamembertForSequenceClassification
from datasets import load_dataset

In [3]:
# Cellule 1
from transformers import CamembertTokenizer, CamembertForSequenceClassification
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from datasets import load_dataset
import evaluate
import numpy as np


In [3]:
# Charger le tokenizer
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")

# Charger le modèle (3 labels : entailment, neutral, contradiction)
model = CamembertForSequenceClassification.from_pretrained("camembert-base", num_labels=3)

print("Tokenizer et modèle chargés !")

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokenizer et modèle chargés !


In [4]:
# Charger XNLI en français
dataset = load_dataset("xnli", "fr")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 392702
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 5010
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 2490
    })
})


In [5]:
def preprocess_function(examples):
    return tokenizer(
        examples["premise"],
        examples["hypothesis"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

tokenized_dataset = dataset.map(preprocess_function, batched=True)
print("Prétraitement terminé !")

Prétraitement terminé !


In [6]:
!pip install --upgrade accelerate transformers --break-system-packages

Defaulting to user installation because normal site-packages is not writeable


In [7]:
!pip install tf-keras --break-system-packages

Defaulting to user installation because normal site-packages is not writeable


In [8]:
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Arguments d'entraînement (noms de paramètres mis à jour)
training_args = TrainingArguments(
    output_dir="./camembert-xnli",
    eval_strategy="epoch",          
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=400,
    per_device_eval_batch_size=400,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_steps=500,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("Trainer prêt !")

/tmp/ipykernel_3816/1477176976.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Trainer prêt !


In [9]:
import torch

# Vérifier si le GPU est disponible
print(f"CUDA disponible : {torch.cuda.is_available()}")
print(f"Nombre de GPU : {torch.cuda.device_count()}")

if torch.cuda.is_available():
    print(f"Nom du GPU : {torch.cuda.get_device_name(0)}")
    
    # Déplacer le modèle sur le GPU
    model = model.to("cuda")
    print("✓ Modèle déplacé sur GPU !")
else:
    print("⚠ Pas de GPU détecté, utilisation du CPU")

CUDA disponible : True
Nombre de GPU : 1
Nom du GPU : Quadro RTX 6000
✓ Modèle déplacé sur GPU !


In [10]:
# Recréer le trainer avec le modèle sur GPU
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Lancer l'entraînement
trainer.train()

/tmp/ipykernel_3816/3329637905.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.701100,0.535849,0.786747
2,0.517200,0.523190,0.791968
3,0.461700,0.518686,0.795984
4,0.421400,0.514604,0.812048
5,0.390400,0.528901,0.804016
6,0.365200,0.552321,0.807631
7,0.344900,0.570326,0.807631
8,0.325600,0.572518,0.811647
9,0.312800,0.584491,0.805221
10,0.302700,0.587133,0.808434


TrainOutput(global_step=9820, training_loss=0.4066727881033406, metrics={'train_runtime': 6730.0356, 'train_samples_per_second': 583.507, 'train_steps_per_second': 1.459, 'total_flos': 2.5831291342155264e+17, 'train_loss': 0.4066727881033406, 'epoch': 10.0})

In [11]:
# Sauvegarder le modèle et le tokenizer
trainer.save_model("./camembert-xnli-final")
tokenizer.save_pretrained("./camembert-xnli-final")

print(" Modèle sauvegardé dans ./camembert-xnli-final")

 Modèle sauvegardé dans ./camembert-xnli-final
